<a href="https://colab.research.google.com/github/justinjunge/Convergent-Wisdom-Project/blob/main/Github_June2024_JJ_Semantic_Search_createEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The original goal of this notebook was to semantically compare any arbitrary sentence to every sentence in the Bible and print the top #X matches.

Jan 13, 2023: This version works for the wikipedia entry on Buddhism

Jan 15, 2023: This version now works for the entire Christian Bible, and generates a model embedding that can be exported and reused for efficiency. (With default settings it takes about 45 minutes to embed).   

Feb 2, 2023: This version was modified to generate a model embedding of the Bhagavad Gita, and is now being modified for the Quran

June 8, 2024: Returning to incomplete embedding of Quran. Note: completed.

June 20: Returning to embedding of Swedenborg's Writings


In [ ]:
# placed at top for manual steps
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
import random
import copy

from google.colab import files

In [ ]:
!pip install -U sentence-transformers
!pip install torch torchvision torchaudio
import torch
!pip install transformers
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
!pip install matplotlib-venn
from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles

In [ ]:
!pip install wikipedia
import wikipedia

!pip install nltk
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

In [ ]:
#model = SentenceTransformer('stsb-roberta-large')
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
# test zone
sentence1 = "William James started the modern field of Psychology as both an empirical and theoretical domain. "
sentence2 = "Skittles are a snack food that is unhealthy."

# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())

In [ ]:
input_phrase = ["There are in general four differing styles in the Word."]
body_text = ["I was told from heaven that there are four special styles of the old Testament.", "The [first is the] style of the ancient Church and of the most ancient, which is like that of the Book of Genesis treating of Paradise, and the tower of Babel."]
# encode list of sentences to get their embeddings
embedding1 = model.encode(input_phrase, convert_to_tensor=True)
embedding2 = model.encode(body_text, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
for i in range(len(input_phrase)):
    for j in range(len(body_text)):
        print("Sentence 1:", input_phrase[i])
        print("Sentence 2:", body_text[j])
        print("Similarity Score:", cosine_scores[i][j].item())
        print()

In [ ]:
text1 = wikipedia.summary("Buddhism")
token_text1 = sent_tokenize(text1)

text2 = wikipedia.summary("Hinduism")
token_text2 = sent_tokenize(text2)

sentences1 = token_text1
sentences2 = token_text2
# encode list of sentences to get their embeddings
embedding1 = model.encode(sentences1, convert_to_tensor=True)
embedding2 = model.encode(sentences2, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
for i in range(len(sentences1)):
    for j in range(len(sentences2)):
        if cosine_scores[i][j].item() > 0.6:
          print("Sentence 1:", sentences1[i])
          print("Sentence 2:", sentences2[j])
          print("Similarity Score:", cosine_scores[i][j].item())
          print()

In [ ]:
input_phrase = ["Meditation is a way to connect more deeply to reality."]
body_text = wikipedia.summary("Buddhism")
body_token = sent_tokenize(body_text)

# encode list of sentences to get their embeddings
embedding1 = model.encode(input_phrase, convert_to_tensor=True)
embedding2 = model.encode(body_token, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

#for i in range(len(input_phrase)):
    #for j in range(len(body_token)):
      #if cosine_scores[i][j].item() > 0.6:
        #print("Sentence 1:", input_phrase[i])
        #print("Sentence 2:", body_text[j])
        #print("Similarity Score:", cosine_scores[i][j].item())
        #print()



In [ ]:
input_phrase = ["Meditation is a way to connect more deeply to reality."]
body_text = wikipedia.page("Buddhism").content
body_token = sent_tokenize(body_text)

# encode list of sentences to get their embeddings
embedding1 = model.encode(input_phrase, convert_to_tensor=True)
embedding2 = model.encode(body_token, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

#for i in range(len(input_phrase)):
    #for j in range(len(body_token)):
      #if cosine_scores[i][j].item() > 0.6:
        #print("Sentence 1:", input_phrase[i])
        #print("Sentence 2:", body_text[j])
        #print("Similarity Score:", cosine_scores[i][j].item())
        #print()

In [ ]:
score_list = list(cosine_scores[0])
#a_list = [10, 11, 14, 23, 9, 3, 35, 22]

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value)
print(max_index)

In [ ]:
score_list[15]

In [ ]:
#df3 = pd.read_csv ('/content/gdrive/MyDrive/jj_Quran.csv')
df3 = pd.read_csv ('/content/gdrive/MyDrive/ncbsw_3a.csv')
print(df3)

In [ ]:
df_Writings2 = df3


In [ ]:
# prompt: Create a new dataframe that contains one entry for each unique chapter appending all of the text in "text" column into one cell per uniquechap

df_Writings_chapter2 = df_Writings2.groupby('uniquechap')['text'].apply(lambda x: ' '.join(x)).to_frame().reset_index()


In [ ]:
df_Writings_chapter2

In [ ]:
df_Quran_chapter["Text"][0]

In [ ]:
# Get the column of interest
column = df_Writings_chapter2["text"]

# Convert the column to a list of strings with quotes
quoted_list = ['"' + str(x) + '"' for x in column]

In [ ]:
Writings_Embedding2 = model.encode(quoted_list, convert_to_tensor=True)

In [ ]:
input_phrase = ["What is the best thing that a person can do?"]
#body_text = wikipedia.page("Buddhism").content
#body_token = sent_tokenize(body_text)
#body_token = quoted_list[0:1000]
# encode list of sentences to get their embeddings
embedding1 = model.encode(input_phrase, convert_to_tensor=True)
#embedding2 = model.encode(body_token, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

#for i in range(len(input_phrase)):
    #for j in range(len(body_token)):
      #if cosine_scores[i][j].item() > 0.6:
        #print("Sentence 1:", input_phrase[i])
        #print("Sentence 2:", body_text[j])
        #print("Similarity Score:", cosine_scores[i][j].item())
        #print()

In [ ]:
len(unique_list)

In [ ]:
#input_phrase = ["What is the meaning of life?"]
body_token = unique_list
# encode list of sentences to get their embeddings
# embedding1 = model.encode(input_phrase, convert_to_tensor=True)  # this is the time intensive step of the entire process
Writings_chapter2 = model.encode(body_token, convert_to_tensor=True)
#print(embeddingQc)

In [ ]:
len(Writings[0])

In [ ]:
input_phrase = ["Live a good and useful life."]

embedding1 = model.encode(input_phrase, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, Writings)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

In [ ]:
input_phrase = ["Where does God live?"]

embedding1 = model.encode(input_phrase, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embeddingQc)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

In [ ]:
input_phrase = ["How should we think about hate?"]

embedding1 = model.encode(input_phrase, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

In [ ]:
input_phrase = ["How should we think about love and enemies?"]

embedding1 = model.encode(input_phrase, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

In [ ]:
input_phrase = ["Don't be afraid"]

embedding1 = model.encode(input_phrase, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

score_list = list(cosine_scores[0])

max_value = max(score_list)
max_index = score_list.index(max_value)

print(max_value, input_phrase, body_token[max_index])

In [ ]:
#import torch

In [ ]:
#embeddingBc = embedding2

In [ ]:
torch.save(Writings_Embedding2, "Writings_Embedding2.pt")

In [ ]:
from google.colab import files

In [ ]:
files.download('Writings_Embedding2.pt')

The code below this includes examples of loading the .csv files with the texts, along with loading the embeddings.

In [ ]:
# The code below will load previously saved embeddings (transformed to vectors), but needs access to the file AND needs a GPU runtime

# embeddingB = torch.load('/content/gdrive/MyDrive/Bible_Embedding.pt', map_location='cuda')

In [ ]:
embeddingG = torch.load('/content/gdrive/MyDrive/Gita3_Embedding.pt', map_location='cuda')
embeddingB = torch.load('/content/gdrive/MyDrive/Bible_Embedding.pt', map_location='cuda')
embeddingQ = torch.load('/content/gdrive/MyDrive/Quran_Embedding.pt', map_location='cuda')

In [ ]:
df_Gita = pd.read_csv ('/content/gdrive/MyDrive/jj_SrimadBhagavadGita.csv')
df_Bible = pd.read_csv ('/content/gdrive/MyDrive/jj_ChristianBible.csv')
df_Quran = pd.read_csv ('/content/gdrive/MyDrive/jj_Quran.csv')
#print(df3)
# Read in the data

# Get the column of interest
columnG = df_Gita["Text"]
columnB = df_Bible["text"]
columnQ = df_Quran["Text"]

# Convert the column to a list of strings with quotes
quoted_listG = ['"' + str(x) + '"' for x in columnG]
quoted_listB = ['"' + str(x) + '"' for x in columnB]
quoted_listQ = ['"' + str(x) + '"' for x in columnQ]

In [ ]:
# Returns top N [num_results] matches to a phrase from the Gita, Bible, & Quran

input_phrase = ["Love everyone and be kind to everyone."]
num_results = 2
embedding1 = model.encode(input_phrase, convert_to_tensor=True)


# compute similarity scores Gita, Bible, Quran
# Gita
cosine_scoresG = util.pytorch_cos_sim(embedding1, embeddingG)
score_listG = list(cosine_scoresG[0])
sorted_listG = sorted(score_listG)
top_NG = sorted_listG[-num_results:]
top_NG = sorted(top_NG, reverse=True)
# Bible
cosine_scoresB = util.pytorch_cos_sim(embedding1, embeddingB)
score_listB = list(cosine_scoresB[0])
sorted_listB = sorted(score_listB)
top_NB = sorted_listB[-num_results:]
top_NB = sorted(top_NB, reverse=True)
# Quran
cosine_scoresQ = util.pytorch_cos_sim(embedding1, embeddingQ)
score_listQ = list(cosine_scoresQ[0])
sorted_listQ = sorted(score_listQ)
top_NQ = sorted_listQ[-num_results:]
top_NQ = sorted(top_NQ, reverse=True)

print(input_phrase)

for i in range(len(top_NG)):
  which_oneG = score_listG.index(top_NG[i])
  print('Gita', top_NG[i], quoted_listG[which_oneG], df_Gita.iloc[which_oneG,0], df_Gita.iloc[which_oneG,1])

for i in range(len(top_NB)):
  which_oneB = score_listB.index(top_NB[i])
  print('Bible', top_NB[i], quoted_listB[which_oneB], df_Bible.iloc[which_oneB,0], df_Bible.iloc[which_oneB,1], df_Bible.iloc[which_oneB,2])

for i in range(len(top_NQ)):
  which_oneQ = score_listQ.index(top_NQ[i])
  print('Quran', top_NQ[i], quoted_listQ[which_oneQ], df_Quran.iloc[which_oneQ,0], df_Quran.iloc[which_oneQ,1])